**Explanation**

Data Preprocessing: Normalize CIFAR-10 images to have values between -1 and 1.
Model Architecture: A simple CNN with:
2 Convolutional layers.

ReLU activations, followed by MaxPooling.
Fully connected layer to predict 10 classes.

Training: Use Cross-Entropy Loss with the Adam optimizer.

Evaluation: Calculate accuracy on the test set.

Notes
Device: For better performance, move model, images, and labels to a GPU if available by setting device = torch.device("cuda" if torch.cuda.is_available() else "cpu").

Hyperparameters: Tweak num_epochs, learning_rate, and batch_size for experimentation.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
import random
from torchvision.models import densenet121

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Hyperparameters
batch_size = 64
num_epochs = 5
learning_rate = 0.001

# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
#Data transformation
# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]) #Normalize

# Data Augumentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train = True, download = True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

# print(train_dataset)
# print(test_dataset)

100%|██████████| 170M/170M [00:02<00:00, 75.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

# print("Train",train_loader)
# print("Test",test_loader)

In [ ]:
#Define CNN Model

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1)#32 filters
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)#64 filters
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.fc1 = nn.Linear(in_features=64*8*8,out_features=128)# Fully connected Layer
        self.fc2 = nn.Linear(in_features=128,out_features=10) # 10classes

    def forward(self,x):
      x = self.pool(torch.relu(self.conv1(x)))
      x = self.pool(torch.relu(self.conv2(x)))
      x = x.view(-1,64*8*8) #flatten
      x = torch.relu(self.fc1(x))
      x = self.fc2(x)
      return x

In [ ]:
class CIFAR10DenseNet(nn.Module):
  def __init__(self):
    super(CIFAR10DenseNet,self).__init__()
    self.densenet = densenet121(pretrained=False)
    self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features,10)
  def forward(self,x):
    return self.densenet(x)

model_densenet = CIFAR10DenseNet().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# Initialize model, loss funtion, and optimizer

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
# print("Model",model)
# print("Model",list(model.parameters()))
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

optimizer = optim.Adam(model_densenet.parameters(), lr=learning_rate)


In this code:

optimizer.zero_grad() ensures that gradients start fresh for each batch.
loss.backward() calculates the gradients.
optimizer.step() updates model parameters using these gradients.

In [ ]:
#trainung function
def train(model,optimizer,loader):
  model.train()  # Set the model to training mode
  train_losses = []
  for epoch in range(num_epochs):
    running_loss =0.0
    for images,labels in loader:  # Iterate over the training data
      optimizer.zero_grad()    # Why we set this?
      outputs = model(images)  #Forward Pass
      loss = criterion(outputs,labels)  #Compute loss
      loss.backward()               # Backpropagation: compute gradients
      optimizer.step()              # Update model parameters
      running_loss += loss.item()   # Accumulate loss
  epoch_loss = running_loss/len(loader)  # Calculate average loss per batch
  train_losses.append(epoch_loss)
  print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")
  return running_loss/len(loader)  # Return the average loss per batch

In [ ]:
#Evaluation Function
def evaluate(model,loader):
  model.eval()  # Set the model to evaluation mode
  correct = 0  # Number of correct predictions
  total = 0  # Total number of predictions
  with torch.no_grad():  # No need to compute gradients during evaluation
    for images,labels in loader:  # Iterate over the test data
      outputs = model(images)  # Forward pass
      _,predicted = torch.max(outputs,1)  # Get the predicted class
      total += labels.size(0)  # Total number of labels
      correct += (predicted == labels).sum().item()  # Number of correct predictions
  accuracy = 100 * correct / total  # Calculate accuracy
  return accuracy  # Return the accuracy


In [ ]:
def train_model(model, optimizer, num_epochs=num_epochs):
    model.train()
    train_losses = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    return train_losses


In [ ]:
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = (correct / total) *100
    return accuracy


In [ ]:
# Training loop

for epoch in range(num_epochs):
    train_loss = train(model, train_loader)
    test_accuracy = evaluate(model, test_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")

print("Training Complete")

Epoch [1/5], Loss: 1.3646, Test Accuracy: 61.60%
Epoch [2/5], Loss: 0.9844, Test Accuracy: 66.83%
Epoch [3/5], Loss: 0.8257, Test Accuracy: 69.36%
Epoch [4/5], Loss: 0.7111, Test Accuracy: 70.59%
Epoch [5/5], Loss: 0.6138, Test Accuracy: 72.26%
Training Complete


In [ ]:
import matplotlib.pyplot as plt

model_metrics= {}

def train_and_record(model,model_name):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses = train_model(model,optimizer)
    accuracies =[]

    for epoch in range(num_epochs):
      accuracy = evaluate_model(model)
      accuracies.append(accuracy)
      print(f"Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy:.2f}%")

    model_metrics[model_name] = {
        'train_losses': train_losses,
        'accuracies': accuracies
    }


In [ ]:
# train each model and record their metrics

train_and_record(SimpleCNN().to(device), 'SimpleCNN')
train_and_record(CIFAR10DenseNet().to(device), 'DenseNet')

Epoch [1/5], Loss: 1.5773
Epoch [2/5], Loss: 1.2688
Epoch [3/5], Loss: 1.1288
Epoch [4/5], Loss: 1.0399
Epoch [5/5], Loss: 0.9777
Epoch [1/5], Accuracy: 66.19%
Epoch [2/5], Accuracy: 65.93%
Epoch [3/5], Accuracy: 66.47%
Epoch [4/5], Accuracy: 65.97%
Epoch [5/5], Accuracy: 65.74%


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/5], Loss: 1.5196
Epoch [2/5], Loss: 1.1371


In [ ]:
# Plotting Training Loss for Each Model

plt.figure(figsize=(10, 6))
for model_name, metrics in model_metrics.items():
    plt.plot(metrics['train_losses'], label = f"{model_name} Train Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss for Each Model')
plt.legend()
plt.show()


#plotting Test Accuracy
plt.figure(figsize=(10, 6))
for model_name, metrics in model_metrics.items():
    plt.plot(metrics['accuracies'], label = f"{model_name} Test Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Test Accuracy for Each Model')
plt.legend()
plt.show()